# Acoustic Binding force maps

### A Sound scattering model based on the method of fundamental solutions


#### Contributors:
*Arnold Kim, Applied Mathematics, University of California, Merced*

*Dustin Kleckner, Physics, University of California, Merced*

*Nicholas St. Clair, Physics (Kleckner Lab), University of California, Merced*

#### Description:
The following notebook makes use of the method of fundamental solutions to compute the sound scattered from multiple sound hard spherical bodies. It determines the forces resultant from said scattering events, some of which are non-linear interactions brought about by interference of acoustic fields. In this specific notebook the data is used to compute the forces between two particles. The location of the second particle is varied, and the force and location data is stored and then plotted as a heat map. This allows one to gauge the structure of the acoustic binding force between two sound hard spherical scatterers at various size parameters.

In [3]:
#Import the relevant python libraries, as well as the MFS master class.

import numpy as np
import scipy.special as sp
import time
import matplotlib.pyplot as plt
import mfs as MFS
print('MFS class downloaded at:', MFS.__file__)
import pandas as pd

MFS class downloaded at: c:\users\nstcl\documents\github\mfs\mfs\__init__.py


## 1. Creating Force Map Data

The following cell sets the relative locations between two acoustically interacting spheres. One of the spheres is held fixed on a pressue node, while the location of the second sphere is varied in the x-z plane. We compute the force on the fixed sphere, and save the x and z components of this force to various arrays, along with the position data. 

In [ ]:
#MFS number
N = 492
#quadrature number
Nq = 8
#wavenumber (1/m)
k = 733
#size parameter
ka = 1
#particle radius
a = ka / k
#Amplitude of incident field
phi_a = 1e-5
#mass density of air
rho = 1.225
#z level of group (m), corresponds to a pressure node
zl = np.pi / (4*k)
#position of first particle
pos1 = [0,0,zl]
#position of second particle
Nx = 100
Nz = 100
x = np.linspace(0.01*a,30/k,Nx)
z = np.linspace(0.01*a,30/k,Nz)
#Initialize empty force data arrays
F_x = np.full((Nx,Nz), 'nan', dtype = 'float')
F_z = np.full((Nx,Nz), 'nan', dtype = 'float')
#Initiate an instance from the master class
scat = MFS.Scatter(k=k, a=a, N=N, Nq=Nq, phi_a = phi_a, lattice_type='icos', rho=rho, source_depth=0.5)
#Call the incoming wave method from the MFS instance, set counterpropogating waves in z
scat.incoming_planewaves([1, 1j], [(0, 0, 1), (0, 0, -1)])

for i in range(Nx):
    
    for j in range(Nz):
        
        #set the position of the second particle
        pos2 = [x[i] , 0 , zl + z[j]]
        
        XD = np.array([pos1,pos2])
        
        #omit regions of space where the two particles overlap
        if np.sqrt(x[i]**2 + z[j]**2) < 2*a:
            F_x[j,i] = 0
            F_z[j,i] = 0
        
        else:
            
            #call the solve method from the master class
            scat.solve(XD)
        
            #save the x and z components of the force on the 1st particle
            #since this particle is always located at a pressure node,
            #we don't have to worry about any single particle effects, i.e. trapping force
            F_x[j,i], F_z[j,i] = scat.force()[0,0],scat.force()[0,2]
            


## 2 Reformatting Data

The following cell simply takes the x and z components of the force data, and computes the radial force on the fixed sphere.

In [ ]:
#The last four lines in this cell save the radial force map data, and accompanying location data, as csv files.
#These lines can be uncommented as needed.

#Compute the reference acoustic force given the incident field strength and the size of the particle
F_0 = np.pi * rho * phi_a**2 * k**2 * a**2
θ = np.arctan(z/x)

#Scale the radial force to the reference force
F_r = (F_x * np.cos(θ) + F_z * np.sin(θ)) / F_0

#df1 = pd.DataFrame(F_r)
#df1.to_csv('Radial_Force_ka=1.csv', header = False, index = False)
#dfx1 = pd.DataFrame(x)
#dfx1.to_csv('xs.csv')

## 3 Visualizing Forces

The following cell plots the radial force between the two spheres as a heatmap, it also shows the overlapping region of the two sphere at the origin of the plot (i.e. where the fixed sphere is located).

In [ ]:
plt.contourf(x,(z+zl), F_r,  cmap = 'RdBu')
plt.gca().add_artist(plt.Circle((0,zl),2*a, color='w', ec='k'))
plt.gca().add_artist(plt.Circle((0,zl),a, color='k'))
plt.colorbar()
plt.gca().set_aspect(1)
plt.xlabel('x (m)')
plt.ylabel('z (m)')
plt.title(f'ka = {ka}')
plt.show()

## 4 Repeat

All the following cells repeat the same scheme outlined above, but for different size parameters (ka).

In [ ]:
#MFS number
N = 492
#quadrature number
Nq = 8
#wavenumber (1/m)
k = 733
#size parameter
ka2 = 2
#particle radius
a2 = ka2 / k
#Amplitude of incident field
phi_a = 1e-5
#mass density of air
rho = 1.225
#z level of group (m)
zl = np.pi / (4*k)
#position of first particle
pos1 = [0,0,zl]
#position of second particle
Nx = 100
Nz = 100
x2 = np.linspace(0.01*a2,30/k,Nx)
z2 = np.linspace(0.01*a2,30/k,Nz)
#Initialize empty force data arrays
F_x2 = np.full((Nx,Nz), 'nan', dtype = 'float')
F_z2 = np.full((Nx,Nz), 'nan', dtype = 'float')
#call MFS scatter class with relevant parameters
scat = MFS.Scatter(k=k, a=a2, N=N, Nq=Nq, phi_a = phi_a, lattice_type='icos', rho=rho, source_depth=0.5)
scat.incoming_planewaves([1, 1j], [(0, 0, 1), (0, 0, -1)])

for i in range(Nx):
    
    for j in range(Nz):
        
        pos2 = [x2[i] , 0 , zl + z2[j]]
        
        XD = np.array([pos1,pos2])
        
        if np.sqrt(x2[i]**2 + z2[j]**2) < 2*a2:
            F_x[j,i] = 0
            F_z[j,i] = 0
        
        else:
        
            scat.solve(XD)
        
            F_x2[j,i], F_z2[j,i] = scat.force()[0,0],scat.force()[0,2]

In [ ]:
F_02 = np.pi * rho * phi_a**2 * k**2 * a2**2
θ2 = np.arctan(z2/x2)
F_r2 = (F_x2 * np.cos(θ2) + F_z2 * np.sin(θ2)) / F_02

df2 = pd.DataFrame(F_r2)
df2.to_csv(f'Radial_Force_ka={ka2}.csv', header = False, index = False)
dfx2 = pd.DataFrame(x2)
dfx2.to_csv('x2s.csv')

In [ ]:
plt.contourf(x2,(z2+zl), F_r2,  cmap = 'RdBu')
plt.gca().add_artist(plt.Circle((0,zl),2*a2, color='w', ec='k'))
plt.gca().add_artist(plt.Circle((0,zl),a2, color='k'))
plt.colorbar()
plt.gca().set_aspect(1)
plt.xlabel('x (m)')
plt.ylabel('z (m)')
plt.title(f'ka = {ka2}')
plt.show()

In [ ]:
#MFS number
N = 492
#quadrature number
Nq = 8
#wavenumber (1/m)
k = 733
#size parameter
ka4 = 4
#particle radius
a4 = ka / k
#Amplitude of incident field
phi_a = 1e-5
#mass density of air
rho = 1.225
#z level of group (m)
zl = np.pi / (4*k)
#position of first particle
pos1 = [0,0,zl]
#position of second particle
Nx = 100
Nz = 100
x4 = np.linspace(0.01*a4,30/k,Nx)
z4 = np.linspace(0.01*a4,30/k,Nz)
#Initialize empty force data arrays
F_x4 = np.full((Nx,Nz), 'nan', dtype = 'float')
F_z4 = np.full((Nx,Nz), 'nan', dtype = 'float')
#call MFS scatter class with relevant parameters
scat = MFS.Scatter(k=k, a=a4, N=N, Nq=Nq, phi_a = phi_a, lattice_type='icos', rho=rho, source_depth=0.5)
scat.incoming_planewaves([1, 1j], [(0, 0, 1), (0, 0, -1)])

for i in range(Nx):
    
    for j in range(Nz):
        
        pos2 = [x4[i] , 0 , zl + z4[j]]
        
        XD = np.array([pos1,pos2])
        
        if np.sqrt(x4[i]**2 + z4[j]**2) < 2*a4:
            F_x[j,i] = 0
            F_z[j,i] = 0
        
        else:
        
            scat.solve(XD)
        
            F_x4[j,i], F_z4[j,i] = scat.force()[0,0],scat.force()[0,2]

In [ ]:
F_04 = np.pi * rho * phi_a**2 * k**2 * a4**2
θ4 = np.arctan(z4/x4)
F_r4 = (F_x4 * np.cos(θ4) + F_z4 * np.sin(θ4)) / F_04
df4 = pd.DataFrame(F_r4)
df4.to_csv(f'Radial_Force_ka={ka4}.csv', header = False, index = False)
dfx4 = pd.DataFrame(x4)
dfx4.to_csv('x4s.csv')

In [ ]:
plt.contourf(x4,(z4+zl), F_r4,  cmap = 'RdBu')
plt.gca().add_artist(plt.Circle((0,zl),2*a4, color='w', ec='k'))
plt.gca().add_artist(plt.Circle((0,zl),a4, color='k'))
plt.colorbar()
plt.gca().set_aspect(1)
plt.xlabel('x (m)')
plt.ylabel('z (m)')
plt.title(f'ka = {ka4}')
plt.show()